In [1]:
include("../path.jl")
include("../ToolBox/ToolBox.jl")
using .ToolBox
using .Geometry

In [2]:
# load meta and pod
master_meta = Load.slc_meta(master_meta_path);
master_precise_orbit = Load.precise_orbit(master_pod_path,master_meta["t_0"]);

In [4]:
burst_number=2

2

In [5]:
meta = master_meta
v_mid_full = mid_burst_speed(master_precise_orbit, meta);
v_mid = v_mid_full[burst_number]

7588.236578903831

In [9]:
samples = round(Int,meta["samples_per_burst"]/2)

12665

In [10]:
    c = 299792458
    k_psi = meta["azimuth_steering_rate"] * pi/180
    dc_coef = meta["burst_meta"]["data_dc_polynomial"][burst_number, :]
    dc_tau0 = meta["burst_meta"]["data_dc_t0"][burst_number]
    fm_coef = meta["burst_meta"]["azimuth_fm_rate_polynomial"][burst_number, :]
    fm_tau0 = meta["burst_meta"]["azimuth_fm_rate_t0"][burst_number]
    f_c = meta["radar_frequency"]
    lines_per_burst = meta["lines_per_burst"]
    number_of_samples = meta["number_of_samples"]
    Delta_t_s = meta["azimuth_time_interval"]
    Delta_tau_s = 1/meta["range_sampling_rate"]
    tau_0 = meta["slant_range_time"]
    v_s = v_mid

    # Temporary functions, allows different x=tau inputs
    k_a(x, fm_param, x0) = fm_param[1] .+ fm_param[2].*(x .- x0) .+ fm_param[3].*(x .- x0).^2 # Doppler FM rate, Eqn. 11
    f_etac(x, dc_param, x0) = dc_param[1] .+ dc_param[2].*(x .- x0) .+ dc_param[3].*(x .- x0).^2; # Doppler centroid freq Eqn. 13

    tau = tau_0 .+ (samples .- 1) .* Delta_tau_s # Slant range time of ith sample, Eqn. 12

    # Doppler rate equations
    k_s = 2 * v_s/c * f_c * k_psi; # Doppler rate from antenna scanning, Eqn. 4
    alpha = 1 .- k_s ./ k_a(tau, fm_coef, fm_tau0); # conversion factor, Eqn. 3
    k_t = k_s ./ alpha; # Doppler Centroid Rate, Eqn. 2

In [17]:
meta["lines_per_burst"]*Delta_t_s

3.132667801199997

In [21]:
k_t*3.132667801199997/2

2388.9699852337694

In [27]:
1/(2388.9699852337694*Delta_t_s)

0.20363851923905543

In [22]:
1/ (pi *k_t*3.132667801199997)

6.662073783916604e-5

In [30]:
1/ (pi *k_t*3.132667801199997) /Delta_t_s *master_meta["azimuth_pixel_spacing"]

0.44879724403478466

In [31]:
6.662073783916604e-5*v_mid

0.5055339197847223